In [1]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('market_data.log'),
        logging.StreamHandler()
    ]
)

In [2]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
import tinkoff.invest
from tinkoff.invest import PortfolioRequest, PortfolioPosition, Client, RequestError, CandleInterval, HistoricCandle, \
    OrderType, OrderDirection, Quotation, InstrumentIdType, InstrumentStatus
from tinkoff.invest.services import InstrumentsService
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/utils')
import run_with_config
importlib.reload(run_with_config)
from run_with_config import run_with_config


2025-04-25 11:28:13,198 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [9]:
sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/data_collection')

#####

import market_data
importlib.reload(market_data)
from market_data import run_pipeline_market

######

import fundamental_data
importlib.reload(fundamental_data)
from fundamental_data import run_pipeline_fundamental

######

import tech_analysis
importlib.reload(tech_analysis)
from tech_analysis import run_pipeline_technical

######

import news_pipeline
importlib.reload(news_pipeline)
from news_pipeline import NewsPipeline

######

import data_integration
importlib.reload(data_integration)
from data_integration import run_pipeline_integration

######

import bonds_processor
importlib.reload(bonds_processor)
from bonds_processor import run_pipeline_bonds

## Globals

In [ ]:
sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/datallection')
from private_info import token, YOUR_API_ID, YOUR_API_HASH, BASE_PATH

kursach_directory_path = BASE_PATH

tickers = pd.read_csv(f"{kursach_directory_path}/data/meta/tickers.csv")
tickers_list = [
    'SBER',  # Сбербанк
    'GAZP',  # Газпром
    'LKOH',  # Лукойл
    'GMKN',  # ГМК "Норильский никель"
    'ROSN',  # Роснефть
    'TATN',  # Татнефть
    'MTSS',  # МТС
    'ALRS',  # АК Алроса
    'SNGS',  # Сургутнефтегаз
    'NVTK',  # Новатэк
    'MVID',  # М.Видео
    'PHOR',  # ФосАгро
    'SIBN',  # Сибнефть
    'AFKS',  # АФК Система
    'MAGN',  # Магнитогорский металлургический комбинат
    'RUAL'   # Русал
]

start_date="2024-01-01"
end_date="2025-04-15"
timeframe="1h"

overall_df = f'{kursach_directory_path}/data/df.csv'

## Initial parser

In [7]:
run_pipeline_market(
    tickers=tickers_list,
    start_date=start_date,
    end_date=end_date,
    token=token
)

2025-04-25 11:36:41 - MarketDataPipeline - INFO - MarketDataPipeline initialized with base directory: /Users/aeshef/Documents/GitHub/kursach/data
2025-04-25 11:36:41,101 - MarketDataPipeline - INFO - MarketDataPipeline initialized with base directory: /Users/aeshef/Documents/GitHub/kursach/data
2025-04-25 11:36:41 - MarketDataPipeline - INFO - Starting market data pipeline for 16 tickers
2025-04-25 11:36:41,103 - MarketDataPipeline - INFO - Starting market data pipeline for 16 tickers
2025-04-25 11:36:41 - MarketDataPipeline - INFO - Date range: 2024-01-01 to 2025-04-15, timeframe: 1min
2025-04-25 11:36:41,104 - MarketDataPipeline - INFO - Date range: 2024-01-01 to 2025-04-15, timeframe: 1min
2025-04-25 11:36:41 - TinkoffDataDownloader - INFO - Initializing instruments from Tinkoff API
2025-04-25 11:36:41,107 - TinkoffDataDownloader - INFO - Initializing instruments from Tinkoff API
2025-04-25 11:36:42,600 - tinkoff.invest.logging - INFO - da242b062f014df20b2683e3abed8367 Shares
2025-0

{'SBER':                       date    open   close     min     max  volume
 0      2024-01-02 07:16:00  268.87  271.04  271.04  268.87      17
 1      2024-01-02 07:17:00  268.87  271.00  271.07  268.87     194
 2      2024-01-02 07:18:00  269.00  270.83  271.00  269.00      94
 3      2024-01-02 07:19:00  269.91  269.91  270.83  269.91      66
 4      2024-01-02 07:20:00  269.91  270.72  270.76  269.91     102
 ...                    ...     ...     ...     ...     ...     ...
 412121 2025-04-14 20:45:00  296.45  296.36  296.46  296.36    1464
 412122 2025-04-14 20:46:00  296.36  296.49  296.49  296.36    1395
 412123 2025-04-14 20:47:00  296.49  296.48  296.50  296.48    1327
 412124 2025-04-14 20:48:00  296.49  296.55  296.55  296.48    1020
 412125 2025-04-14 20:49:00  296.54  296.50  296.55  295.83    8854
 
 [412126 rows x 6 columns],
 'GAZP':                       date    open   close     min     max  volume
 0      2024-01-02 07:16:00  160.34  159.40  160.34  159.40      22
 1

## Fundamental

In [10]:
run_pipeline_fundamental(ticker_list=tickers_list)

2025-04-25 11:45:34 - FundamentalPipeline - INFO - Processing tickers: ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL']
2025-04-25 11:45:34,554 - FundamentalPipeline - INFO - Processing tickers: ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL']
2025-04-25 11:45:34 - FundamentalPipeline - INFO - Fetching data for SBER
2025-04-25 11:45:34,558 - FundamentalPipeline - INFO - Fetching data for SBER
2025-04-25 11:45:35 - FundamentalPipeline - INFO - Successfully processed SBER
2025-04-25 11:45:35,050 - FundamentalPipeline - INFO - Successfully processed SBER
2025-04-25 11:45:35 - FundamentalPipeline - INFO - Fetching data for GAZP
2025-04-25 11:45:35,051 - FundamentalPipeline - INFO - Fetching data for GAZP
2025-04-25 11:45:35 - FundamentalPipeline - INFO - Successfully processed GAZP
2025-04-25 11:45:35,777 - FundamentalPipeline - INF

## Tech

In [11]:
run_pipeline_technical()

2025-04-25 11:46:08 - TechAnalysisPipeline - INFO - === НАЧАЛО ТЕХНИЧЕСКОГО АНАЛИЗА ===
2025-04-25 11:46:08,142 - TechAnalysisPipeline - INFO - === НАЧАЛО ТЕХНИЧЕСКОГО АНАЛИЗА ===
2025-04-25 11:46:08 - TechAnalysisPipeline - INFO - Обработка тикера: GAZP
2025-04-25 11:46:08,145 - TechAnalysisPipeline - INFO - Обработка тикера: GAZP
2025-04-25 11:46:08 - TechAnalysisPipeline - INFO - Используем файл: GAZP.parquet
2025-04-25 11:46:08,147 - TechAnalysisPipeline - INFO - Используем файл: GAZP.parquet
2025-04-25 11:46:08 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:08,149 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:08 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-25 11:46:08,195 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-25 11:46:08 - TechAnalysisPipeline - INFO - Сохранен CSV с индикаторами: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-25 11:46:08 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_MACD.png
2025-04-25 11:46:08,613 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/GAZP/tech_analysis/GAZP_MACD.png
2025-04-25 11:46:08 - TechAnalysisPipeline - INFO - Обработка тикера: SBER
2025-04-25 11:46:08,614 - TechAnalysisPipeline - INFO - Обработка тикера: SBER
2025-04-25 11:46:08 - TechAnalysisPipeline - INFO - Используем файл: SBER.parquet
2025-04-25 11:46:08,615 - TechAnalysisPipeline - INFO - Используем файл: SBER.parquet
2025-04-25 11:46:08 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:08,615 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:08 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-25 11:46:08,712 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
20

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-25 11:46:09 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_MACD.png
2025-04-25 11:46:09,124 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/SBER/tech_analysis/SBER_MACD.png
2025-04-25 11:46:09 - TechAnalysisPipeline - INFO - Обработка тикера: LKOH
2025-04-25 11:46:09,125 - TechAnalysisPipeline - INFO - Обработка тикера: LKOH
2025-04-25 11:46:09 - TechAnalysisPipeline - INFO - Используем файл: LKOH.parquet
2025-04-25 11:46:09,126 - TechAnalysisPipeline - INFO - Используем файл: LKOH.parquet
2025-04-25 11:46:09 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:09,127 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:09 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-25 11:46:09,157 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
20

...

2025-04-25 11:46:14 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/AFKS/tech_analysis/AFKS_MACD.png
2025-04-25 11:46:14,676 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/AFKS/tech_analysis/AFKS_MACD.png
2025-04-25 11:46:14 - TechAnalysisPipeline - INFO - Обработка тикера: MAGN
2025-04-25 11:46:14,677 - TechAnalysisPipeline - INFO - Обработка тикера: MAGN
2025-04-25 11:46:14 - TechAnalysisPipeline - INFO - Используем файл: MAGN.parquet
2025-04-25 11:46:14,678 - TechAnalysisPipeline - INFO - Используем файл: MAGN.parquet
2025-04-25 11:46:14 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:14,678 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:14 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-25 11:46:14,706 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
20

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-25 11:46:15 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_MACD.png
2025-04-25 11:46:15,089 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/MAGN/tech_analysis/MAGN_MACD.png
2025-04-25 11:46:15 - TechAnalysisPipeline - INFO - Обработка тикера: RUAL
2025-04-25 11:46:15,090 - TechAnalysisPipeline - INFO - Обработка тикера: RUAL
2025-04-25 11:46:15 - TechAnalysisPipeline - INFO - Используем файл: RUAL.parquet
2025-04-25 11:46:15,091 - TechAnalysisPipeline - INFO - Используем файл: RUAL.parquet
2025-04-25 11:46:15 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:15,091 - TechnicalIndicators - INFO - Начало загрузки данных.
2025-04-25 11:46:15 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
2025-04-25 11:46:15,116 - TechnicalIndicators - INFO - Начало расчёта технических индикаторов.
20

Index(['date', 'open', 'close', 'low', 'high', 'volume'], dtype='object')


2025-04-25 11:46:15 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_MACD.png
2025-04-25 11:46:15,509 - TechAnalysisPipeline - INFO - Сохранен график MACD: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/RUAL/tech_analysis/RUAL_MACD.png
2025-04-25 11:46:15 - TechAnalysisPipeline - INFO - Сводный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/tech_analysis_summary.txt
2025-04-25 11:46:15,510 - TechAnalysisPipeline - INFO - Сводный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/tech_analysis_summary.txt
2025-04-25 11:46:15 - TechAnalysisPipeline - INFO - === ТЕХНИЧЕСКИЙ АНАЛИЗ ЗАВЕРШЕН ===
2025-04-25 11:46:15,511 - TechAnalysisPipeline - INFO - === ТЕХНИЧЕСКИЙ АНАЛИЗ ЗАВЕРШЕН ===


## News

In [ ]:
run_with_config(
    script_path=f"{kursach_directory_path}/pys/data_collection/news_pipeline.py",
    class_or_function="NewsPipeline",
    method="run_pipeline",
    config_filename="news_pipeline_config.json",
    
    tickers=tickers_list,
    collect_telegram=True,
    telegram_api_id=YOUR_API_ID,
    telegram_api_hash=YOUR_API_HASH,
    telegram_channel="cbrstocks",
    telegram_limit=10000,
    start_date=start_date,
    end_date=end_date,
    use_cached_telegram=True
)

Конфигурация сохранена в /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск команды: /Library/Developer/CommandLineTools/usr/bin/python3 /Users/aeshef/Documents/GitHub/kursach/pys/data_collection/news_pipeline.py --config /Users/aeshef/Documents/GitHub/kursach/news_pipeline_config.json
Запуск pipeline.run_pipeline с параметрами: {'tickers': ['SBER', 'GAZP', 'LKOH', 'GMKN', 'ROSN', 'TATN', 'MTSS', 'ALRS', 'SNGS', 'NVTK', 'MVID', 'PHOR', 'SIBN', 'AFKS', 'MAGN', 'RUAL'], 'collect_telegram': True, 'telegram_api_id': 28994010, 'telegram_api_hash': '6e7a57fdfe1a10b0a3434104b42badf2', 'telegram_channel': 'cbrstocks', 'telegram_limit': 10000, 'start_date': datetime.date(2024, 1, 1), 'end_date': datetime.date(2025, 4, 15), 'use_cached_telegram': True}
2025-04-23 02:56:50 - NewsPipeline - INFO - Запуск пайплайна анализа новостей для 16 тикеров
2025-04-23 02:56:50 - NewsPipeline - INFO - Диапазон дат: 2024-01-01 - 2025-04-15
2025-04-23 02:56:50 - pymorphy2.opencorpora_dict.wr

0

In [ ]:
run_pipeline_integration(tickers=tickers_list)

2025-04-23 03:02:44,872 - DataIntegrator - INFO - Обработка тикера SBER...
2025-04-23 03:02:44,918 - DataIntegrator - INFO - Данные для SBER загружены за период: 2024-01-02 - 2025-04-14
2025-04-23 03:02:44,920 - DataIntegrator - INFO -   - Обнаружено 5400 пропущенных значений для SBER
2025-04-23 03:02:45,011 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-23 03:02:45,012 - DataIntegrator - INFO - Обработка тикера GAZP...
2025-04-23 03:02:45,038 - DataIntegrator - INFO - Данные для GAZP загружены за период: 2024-01-02 - 2025-04-14
2025-04-23 03:02:45,039 - DataIntegrator - INFO -   - Обнаружено 5177 пропущенных значений для GAZP
2025-04-23 03:02:45,131 - DataIntegrator - INFO -   - Пропущенные значения заполнены методом zero
2025-04-23 03:02:45,133 - DataIntegrator - INFO - Обработка тикера LKOH...
2025-04-23 03:02:45,156 - DataIntegrator - INFO - Данные для LKOH загружены за период: 2024-01-02 - 2025-04-14
2025-04-23 03:02:45,157 - DataIntegrator - INF

In [12]:
bonds_results=run_pipeline_bonds(
    # Ожидая снижение ставки, увеличиваем вес доходности
    score_weights={
        'yield': 0.7,           # Больше вес на доходность
        'duration': 0.2,        # Меньше вес на дюрацию
        'tax_benefit': 0.05,    # Меньше вес на налоговую льготу
        'price_discount': 0.05  # Меньше вес на скидку
    },
    market_condition='high_rate',
    select_top_n=5,
    date_range=(start_date, end_date)
)

risk_free_rate = bonds_results['risk_free_rate']


2025-04-25 11:46:39,249 - bonds_pipeline - INFO - Запуск пайплайна обработки облигаций в 2025-04-25 11:46:39
2025-04-25 11:46:39,251 - bonds_pipeline - INFO - Начинаем загрузку и обработку данных по облигациям из /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/csv
2025-04-25 11:46:39,254 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-03-26.csv с 19 облигациями от 2024-03-26
2025-04-25 11:46:39,257 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-04-10.csv с 20 облигациями от 2024-04-10
2025-04-25 11:46:39,259 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-05-24.csv с 15 облигациями от 2024-05-24
2025-04-25 11:46:39,261 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-08-09.csv с 27 облигациями от 2024-08-09
2025-04-25 11:46:39,263 - bonds_pipeline - INFO - Успешно загружен файл bond_search_2024-08-10.csv с 30 облигациями от 2024-08-10
2025-04-25 11:46:39,265 - bonds_pipeline - INFO - Успешно загружен ф


Топ рекомендуемых облигаций:
    security_code                  full_name  yield  duration_years  \
336  RU000A105MP6  ДОМОДЕДОВО ФьюэлФ 002Р-01  37.19        0.805000   
352  RU000A106HF5     ЭЛЕКТРОРЕШЕНИЯ 001Р-01  32.58        1.277500   
347  RU000A104ZK2          МВ ФИНАНС 001Р-03  29.83        0.421667   
343  RU000A104YT6          ГК Самолет БО-П12  29.78        0.391667   
335  RU000A10ARS4          МОНОПОЛИЯ 001P-03  28.89        0.841667   

     risk_adjusted_yield  bond_score  weight  
336            46.198758    2.237186  0.1526  
352            25.502935    1.442737  0.0962  
347            70.743083    1.190934  0.2914  
343            76.034043    1.183059  0.3137  
335            34.324752    0.868785  0.1460  
Обработано 24 записей облигаций
Текущая безрисковая ставка: 25.49%

Топ рекомендуемых облигаций:
    security_code                  full_name  yield  duration_years  \
336  RU000A105MP6  ДОМОДЕДОВО ФьюэлФ 002Р-01  37.19        0.805000   
352  RU000A106HF5     

<!-- я делаю курсовой проект. я придерживаюсь этого плана:
# ---  
## 1. Слой Сбора и Агрегации Данных

### 1.1. Рыночные данные
- Источники API: Yahoo Finance, Alpha Vantage для акций. Для облигаций/ОФЗ — выгрузки или ручная подготовка, если нет автосбора.
- Загрузка исторических котировок всех бумаг, которые потенциально могут войти в портфель (акции + risk-free/фонды).
- Сохранение единой “сырых” данных (CSV/Parquet).

### 1.2. Фундаментальные данные
- Используются только последние доступные отчёты по каждой бумаге (например, за 2023–24, чаще нет).
- Допускается ручной ввод/подготовка csv коллегой (дефолтно — только 1 snapshot на тикер).
- Промышленный подход: между апдейтами отчётов — “forward fill” фундаментальных коэффициентов; при вычислениях делается анализ только cross-sectional (по состоянию на одну дату между бумагами).
- Стандартизация (z-score) коэффициентов для последующего скоринга.

### 1.3. Новостной/сентимент-поток
- Автоматизация выгрузки новостей (RSS, Twitter, базовые каналы для акций и risk-free).
- Анализ новостного сентимента (проще — ручная классификация “важные макро/кризисы” для risk-free бумаг).
- Сохранение текстовых и сентиментных фич в отдельные поля.

---

## 2. Слой Предобработки и Признаков

### 2.1. Генерация признаков для акций и risk-free
- Чистка рядов; заполнение пропусков.
- Генерация технических индикаторов (SMA, EMA, RSI, MACD, Bollinger Bands, паттерны — минимум сигналы типа "пересечение SMA").
- Для risk-free инструментов — расчет доходностей/дюраций “на сейчас”, если есть хотя бы скриншот доходности.
- Для всех сегментов — доходности, волатильности, ковариации.

### 2.2. Работа с фундаменталом (ограничена snapshot-режимом!)
- Для единственного года: сравнение бумаг между собой — расчёт z-score фундаментальных метрик на дату.
- Проведение статического ранжирования (квантили, top/bottom группы).
- Генерация сводного композитного score для отбора universe.

### 2.3. Интеграция сентимента и фундаментала
- Корректировка статичного funda-score с учётом свежих новостных сигналов (например, большой рынок негатив — понизить вес funda-score).
- Для risk-free — влияние только “макро”-новостей и текущей ставочной среды.

---

## 3. Модуль анализа и сигналов

### 3.1. Генерация торговых сигналов
- Акции: Buy/Hold/Sell по композитному score (технические + функдаментал + новости).
- Для risk-free: всегда Hold/Buy (core allocation), исключения — при негативных макро/ставках.

### 3.2. Кросс-секционный квантильный отбор
- “Shortlist” бумаг для портфеля: top 30–50% по funda-score, приоритизированных техническими индикаторами.
- Для risk-free инструментов можно просто фиксировать веса или задавать диапазон для оптимизации.

---  

## 4. Оптимизация Портфеля

### 4.1. Модель Марковица (Mean-Variance)
- Оптимизация среди shortlist бумаг (только Buy/Hold).
- Явно фиксируем вес на risk-free (“core allocation”, например 25–35%).
- Остальное распределяем по акциям оптимально по риск-доходности (mean-variance).

### 4.2. Black-Litterman (опционально)
- Возможность внести “view” на основе ранга funda-score и новостных сигналов.
- Коррекция “базовых” рыночных весов с помощью своих score.

---

## 5. Интеграция, отчётность и автоматизация

### 5.1. Финальные сигналы и обработка портфеля
- Взвешенная агрегация сигналов (funda-score × техника × новости).
- Единый пайплайн: загрузка — признаки — сигналы — оптимизация — результат.
- Пайплайны собраны как большие скрипты-автоматизаторы для (квази)ежедневного обновления.

### 5.2. Интерфейс пользователя
- Просмотр результатов
по кнопке (CLI/Jupyter/Telegram-бот).
- Возможность быстро изменить параметры universe/core allocation.

---

## 6. Бэктестинг и визуализация

### 6.1. Бэктест
- Проверка стратегии на годичном горизонте (по доступным данным).
- Особое внимание: для фундаментала — тестирование, как бы работали “портфели top vs bottom квантилей по snapshot”.
- Акцент на анализ разницы “просто держать весь рынок” vs. “отбирать по фунду/score”.

### 6.2. Визуализация
- Графики динамики портфеля, групп, эффективная граница.
- Визуализация сравнения групп, влияния сигналов.

---

## 7. Автоматизация и пайплайны

- Все ключевые этапы реализованы как отдельные большие py-скрипты (классы: DataCollector, FeatureEngineer, SignalGenerator, PortfolioOptimizer, Backtester, Visualizer).
- Каждый пайплайн сохраняет промежуточные дампы для отладки.
- Финальный мастер-пайплайн (“одна кнопка”).
- Делегирование ручных элементов коллегой: подготовка funda/csv, ручная разметка событий/новостей, первичный запуск функций.

---

## 8. Особенности и работа с мало меняющимися фундаментальными данными

- Прямо указывается в отчёте: фундаментальные признаки “медленные”, обновляются редко, анализ кросс-секционный: используется только сравнение между бумагами на одну дату, либо квантилизация и ранжирование при каждом отчёте.
- Расширение базы данных даже на 2–3 года (если возможно) — значительно повысит качество анализа и возможности rolling backtest.
- Все краткосрочные сигналы (техникa, новости) агрегируются для “ускорения” pipeline решений между публикациями отчетов.

---

## 9. Как преподнести в работе/отчёте

> Фундаментальные данные используются в cross-sectional анализе и квантилизации портфелей из-за своей низкой частоты обновления и ограниченной истории. Это соответствует практике индустрии для развивающихся рынков/ограниченных источников. При появлении новых отчетов pipeline автоматически обновляет ранжирование бумаг и состава портфеля. Краткосрочные решения строятся преимущественно на технических и новостных сигналах.

---

## Если понадобится — скину шаблоны py-файлов и классов по этому плану.


на данный момент я нахожусь на таком этапе: 
1) структура проекта
/Users/aeshef/Documents/GitHub/kursach
(├── df.csv - обьединенный датасет с 384 фичами для 17 тикеров за 2024-2025 год
├── figi.txt
├── news_analysis_summary.txt
├── processed_data
│   ├── AFKS
│   │   ├── AFKS.parquet
│   │   ├── AFKS_all_news_processed.csv
│   │   ├── AFKS_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── AFKS_daily_events.csv
│   │   │   ├── AFKS_daily_sentiment.csv
│   │   │   ├── AFKS_event_distribution.png
│   │   │   ├── AFKS_ml_features.csv
│   │   │   ├── AFKS_news_with_events.csv
│   │   │   ├── AFKS_news_with_sentiment.csv
│   │   │   ├── AFKS_price_vs_sentiment.png
│   │   │   ├── AFKS_sentiment_distribution.png
│   │   │   ├── AFKS_sentiment_dynamics.png
│   │   │   ├── AFKS_sentiment_features.csv
│   │   │   └── AFKS_topics.txt
│   │   └── tech_analysis
│   │       ├── AFKS_MACD.png
│   │       ├── AFKS_price_indicators.png
│   │       └── AFKS_tech_indicators.csv
...
│   ├── VTBR
│   │   ├── VTBR.parquet
│   │   ├── VTBR_all_news_processed.csv
│   │   ├── VTBR_news_20250422_1601.csv
│   │   ├── news_analysis
│   │   │   ├── VTBR_daily_events.csv
│   │   │   ├── VTBR_daily_sentiment.csv
│   │   │   ├── VTBR_event_distribution.png
│   │   │   ├── VTBR_ml_features.csv
│   │   │   ├── VTBR_news_with_events.csv
│   │   │   ├── VTBR_news_with_sentiment.csv
│   │   │   ├── VTBR_price_vs_sentiment.png
│   │   │   ├── VTBR_sentiment_distribution.png
│   │   │   ├── VTBR_sentiment_dynamics.png
│   │   │   ├── VTBR_sentiment_features.csv
│   │   │   └── VTBR_topics.txt
│   │   └── tech_analysis
│   │       ├── VTBR_MACD.png
│   │       ├── VTBR_price_indicators.png
│   │       └── VTBR_tech_indicators.csv
│   ├── bonds_csv
│   │   ├── bond_search_2023-12-01.csv
│   │   ├── bond_search_2024-03-26.csv
│   │   ├── bond_search_2024-04-10.csv
│   │   ├── bond_search_2024-05-24.csv
│   │   ├── bond_search_2024-08-09.csv
│   │   ├── bond_search_2024-08-10.csv
│   │   ├── bond_search_2024-08-12.csv
│   │   ├── bond_search_2024-09-04.csv
│   │   ├── bond_search_2024-09-13.csv
│   │   ├── bond_search_2024-10-30.csv
│   │   ├── bond_search_2024-11-26.csv
│   │   ├── bond_search_2024-12-04.csv
│   │   ├── bond_search_2025-01-28.csv
│   │   ├── bond_search_2025-02-10.csv
│   │   ├── bond_search_2025-02-14.csv
│   │   └── bond_search_2025-04-22.csv
...
├── raw_data
│   ├── AFKS
│   │   ├── BBG004S68614_2024
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240102.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240103.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240104.csv
│   │   │   ├── 53b67587-96eb-4b41-8e0c-d2e3c0bdd234_20240105.csv
...

├── status.json
├── telegram_news
│   ├── AFKS_telegram_news_20250422.csv
│   ├── ALRS_telegram_news_20250422.csv
│   ├── GAZP_telegram_news_20250422.csv
│   ├── GMKN_telegram_news_20250422.csv
│   ├── LKOH_telegram_news_20250422.csv
│   ├── MAGN_telegram_news_20250422.csv
│   ├── MTSS_telegram_news_20250422.csv
│   ├── MVID_telegram_news_20250422.csv
│   ├── NVTK_telegram_news_20250422.csv
│   ├── PHOR_telegram_news_20250422.csv
│   ├── ROSN_telegram_news_20250422.csv
│   ├── RUAL_telegram_news_20250422.csv
│   ├── SBER_telegram_news_20250422.csv
│   ├── SIBN_telegram_news_20250422.csv
│   ├── SNGS_telegram_news_20250422.csv
│   ├── TATN_telegram_news_20250422.csv
│   ├── VTBR_telegram_news_20250422.csv
│   └── telegram_headlines_20250422_160151.csv
└── tickers.csv) - конец папки data
├── ipynbs
│   ├── market_data.log
│   └── overall_data_pipeline.ipynb - тут в виде ноутбука (потому будет py файлом) расписан полный пайплайн сбора тикеров, тех анализа, фундаментального анализа, извлечения фичей из новостей и получение объединенного датасета со всеми фичами для всех тикеров (df.csv)
├── news_pipeline_config.json
├── pys
│   ├── __pycache__
│   │   ├── event_detector.cpython-313.pyc
│   │   ├── news_feature_extractor.cpython-313.pyc
│   │   ├── news_integration.cpython-313.pyc
│   │   ├── news_preprocessor.cpython-313.pyc
│   │   ├── news_visualizer.cpython-313.pyc
│   │   └── sentiment_analysis.cpython-313.pyc
│   ├── data_integration.py
│   ├── event_detector.py
│   ├── fundamental_data.py
│   ├── market_data.py
│   ├── news_feature_extractor.py
│   ├── news_integration.py
│   ├── news_pipeline.py
│   ├── news_preprocessor.py
│   ├── news_visualizer.py
│   ├── portfolio_optimizer.py
│   ├── run_with_config.py
│   ├── sentiment_analysis.py
│   ├── tech_analysis.py
│   └── technical_indicators.py
└── telegram_session.session

2) я еще вообще никак не работал с данными bonds, так что с ними что-то надо сделать. 

у меня есть 7 дней на выполнение всего курсового проекта, так что сейчас от тебя мне нужно много кода. предложи в соответствии с планом, что мне стоит сейчас сделать -->

In [21]:
sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/porfolio_optimization')

#####

import signal_generator
importlib.reload(signal_generator)
from signal_generator import run_pipeline_signal_generator

#####

import portfolio_optimizer
importlib.reload(portfolio_optimizer)
from portfolio_optimizer import PortfolioOptimizer

#####

import backtester
importlib.reload(backtester)
from backtester import Backtester

In [22]:
run_pipeline_signal_generator(
    weight_tech=0.5,
    weight_sentiment=0.3,
    weight_fundamental=0.2
)

2025-04-25 13:08:39,412 - signal_generator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-25 13:08:39,412 - signal_generator - INFO - Запуск пайплайна генерации торговых сигналов
2025-04-25 13:08:39,416 - signal_generator - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/df.csv
2025-04-25 13:08:39,416 - signal_generator - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/df.csv
2025-04-25 13:08:39,596 - signal_generator - INFO - Загружено 7471 строк данных
2025-04-25 13:08:39,596 - signal_generator - INFO - Загружено 7471 строк данных
2025-04-25 13:08:39,596 - signal_generator - INFO - Расчет композитного скора
2025-04-25 13:08:39,596 - signal_generator - INFO - Расчет композитного скора
2025-04-25 13:08:39,602 - signal_generator - INFO - Технический скор рассчитан на основе 6 индикаторов
2025-04-25 13:08:39,602 - signal_generator - INFO - Технический скор рассчитан на основе 6 индикаторов
2025-04-25 13:08:39,605 - signal_generat

In [23]:
portfolio = PortfolioOptimizer(
    risk_free_rate=risk_free_rate,
    min_rf_allocation=0.3,
    max_rf_allocation=0.5
).run_pipeline()

2025-04-25 13:08:55,514 - portfolio_optimizer - INFO - Запуск пайплайна оптимизации портфеля
2025-04-25 13:08:55,514 - portfolio_optimizer - INFO - Запуск пайплайна оптимизации портфеля
2025-04-25 13:08:55,516 - portfolio_optimizer - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-25 13:08:55,516 - portfolio_optimizer - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-25 13:08:55,699 - portfolio_optimizer - INFO - Загружено 7471 строк данных
2025-04-25 13:08:55,699 - portfolio_optimizer - INFO - Загружено 7471 строк данных
2025-04-25 13:08:55,700 - portfolio_optimizer - INFO - Подготовка данных доходностей для оптимизации
2025-04-25 13:08:55,700 - portfolio_optimizer - INFO - Подготовка данных доходностей для оптимизации
2025-04-25 13:08:55,704 - portfolio_optimizer - INFO - Отфильтровано 7030 строк по shortlist и сигналам
2025-04-25 13:08:55,704 - portfolio_optimizer - INFO - Отфильтровано 7030 строк по

In [16]:
Backtester(
    portfolio_weights=portfolio['weights'],
    start_date=start_date,
    end_date=end_date
).run_pipeline()

2025-04-25 11:48:58,576 - backtester - INFO - Запуск пайплайна бэктестирования
2025-04-25 11:48:58,576 - backtester - INFO - Запуск пайплайна бэктестирования
2025-04-25 11:48:58,577 - backtester - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-25 11:48:58,577 - backtester - INFO - Загрузка данных из /Users/aeshef/Documents/GitHub/kursach/data/signals.csv
2025-04-25 11:48:58,778 - backtester - INFO - Загружено 7471 строк данных
2025-04-25 11:48:58,778 - backtester - INFO - Загружено 7471 строк данных
2025-04-25 11:48:58,785 - backtester - INFO - Период бэктеста: 2024-01-02 - 2025-04-14
2025-04-25 11:48:58,785 - backtester - INFO - Период бэктеста: 2024-01-02 - 2025-04-14
2025-04-25 11:48:58,786 - backtester - INFO - Расчет доходностей по тикерам
2025-04-25 11:48:58,786 - backtester - INFO - Расчет доходностей по тикерам
2025-04-25 11:48:58,801 - backtester - INFO - Рассчитаны доходности для 16 тикеров
2025-04-25 11:48:58,801 - backtester - INFO

{'metrics': {'cumulative_return': 0.03818641301080494,
  'annual_return': 0.020384011722720752,
  'annual_volatility': 0.025138251866773927,
  'sharpe_ratio': -297.53922539710646,
  'max_drawdown': -0.018332517106930624,
  'win_rate': 0.7970085470085471},
 'returns':                 AFKS          ALRS      GAZP          GMKN          LKOH  \
 date                                                                       
 2024-01-02       NaN           NaN       NaN           NaN           NaN   
 2024-01-03 -0.000214 -1.112135e-14 -0.000786 -3.614488e-15 -2.993974e-15   
 2024-01-04 -0.000632 -1.103592e-16  0.000141  1.707925e-15  1.416641e-15   
 2024-01-05  0.000753 -1.379102e-15 -0.000513  4.571237e-16 -3.941836e-16   
 2024-01-06 -0.000037  3.298267e-16  0.000067  4.003763e-16 -6.561006e-17   
 ...              ...           ...       ...           ...           ...   
 2025-04-10 -0.000000 -0.000000e+00  0.000000 -0.000000e+00  5.993132e-15   
 2025-04-11  0.000000  0.000000e+00  0.0